In [1]:
import time
import tensorflow as tf
from keras import Sequential
import keras.utils.np_utils as kutils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout

In [2]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        'data/train',
        target_size=(64, 64),
        batch_size=20000,
        class_mode='binary')


test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        'data/test',
        target_size=(64, 64),
        batch_size=2500,
        class_mode='binary')

print("Classes of data: ", train_generator.class_indices)
data_train = train_generator.next()
data_test = test_generator.next()

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Classes of data:  {'cats': 0, 'dogs': 1}


In [3]:
(x_train, y_train) = data_train
(x_test, y_test) = data_test

In [4]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy',  metrics=['accuracy'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 62, 62, 32)       128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 31, 31, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 29, 29, 64)       2

In [6]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=8)

In [7]:
start_time = time.time()

history = model.fit(x_train, y_train, epochs=100, batch_size=100, verbose=1, validation_split=0.1, callbacks=[early_stopping])

print("--- %s seconds ---" % (time.time() - start_time))

loss = history.history['loss']

Epoch 1/100


2022-11-29 11:54:29.641316: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


180/180 [==============================] - 29s 159ms/step - loss: 0.7386 - accuracy: 0.6467 - val_loss: 1.8505 - val_accuracy: 0.5070
Epoch 2/100
180/180 [==============================] - 29s 161ms/step - loss: 0.5474 - accuracy: 0.7282 - val_loss: 0.9862 - val_accuracy: 0.5355
Epoch 3/100
180/180 [==============================] - 30s 165ms/step - loss: 0.4819 - accuracy: 0.7691 - val_loss: 0.5740 - val_accuracy: 0.7220
Epoch 4/100
180/180 [==============================] - 29s 160ms/step - loss: 0.4428 - accuracy: 0.7940 - val_loss: 0.9076 - val_accuracy: 0.6565
Epoch 5/100
180/180 [==============================] - 29s 162ms/step - loss: 0.4181 - accuracy: 0.8055 - val_loss: 0.5618 - val_accuracy: 0.7240
Epoch 6/100
180/180 [==============================] - 29s 161ms/step - loss: 0.3870 - accuracy: 0.8245 - val_loss: 0.4538 - val_accuracy: 0.7950
Epoch 7/100
180/180 [==============================] - 29s 159ms/step - loss: 0.3691 - accuracy: 0.8334 - val_loss: 0.6023 - val_accurac

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
loss = history.history['loss']
plt.plot(loss)

In [ ]:
from IPython.display import Image
Image(filename='loss_graph_Generic_0_front.png') 

# Regular Model Accuracy

In [ ]:
result = model.evaluate(x_test, y_test)
print("Accuracy:", result[1]*100, "%")

# Federated Model Accuracy

In [ ]:
import joblib
model = joblib.load("Generic.model")

result = model.evaluate(x_test, y_test)
print("Accuracy:", result[1]*100, "%")

In [ ]:
x_train.shape

In [ ]:
a = x_train.reshape(10,2000,32,32,3)

In [ ]:
a.shape

In [ ]:
a[0].shape